In [ ]:
!pip install xgboost
!pip install lightgbm
import numpy as np 
import pandas as pd 
import seaborn as sns
import re
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from warnings import filterwarnings
filterwarnings('ignore')
from scipy.stats.mstats import winsorize
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error,explained_variance_score

from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.ensemble import (RandomForestRegressor)
from sklearn.model_selection import (GridSearchCV, cross_val_score, cross_val_predict,
                                     StratifiedKFold, learning_curve)

from statsmodels.tools.eval_measures import mse, rmse
from sklearn import preprocessing
from sklearn.svm import SVR
import xgboost
import lightgbm as lgb
from xgboost import XGBRegressor



**Analyzing DataSet**

In [ ]:
df = pd.read_csv('../input/life-expectancy-who/Life Expectancy Data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.columns.values

In [ ]:
df.info() # Only Country and Status are categorical variables.

**EDA**

In [ ]:
#Fixing column names
df.rename(columns = {" BMI " :"BMI", 
                                  "Life expectancy ": "Life_expectancy",
                                  "Adult Mortality":"Adult_mortality",
                                  "infant deaths":"Infant_deaths",
                                  "percentage expenditure":"Percentage_expenditure",
                                  "Hepatitis B":"HepatitisB",
                                  "Measles ":"Measles",
                                  "under-five deaths ": "Under_five_deaths",
                                  "Total expenditure":"Total_expenditure",
                                  "Diphtheria ": "Diphtheria",
                                  " thinness  1-19 years":"Thinness_1-19_years",
                                  " thinness 5-9 years":"Thinness_5-9_years",
                                  " HIV/AIDS":"HIV/AIDS",
                                  "Income composition of resources":"Income_composition_of_resources"}, inplace = True)

In [ ]:
df.nunique(axis=0)

In [ ]:
df.describe().T

**Correlation between Features**

In [ ]:
correlation = df.corr()
print(correlation)

In [ ]:
plt.figure(figsize=(18,14))
sns.heatmap(correlation,annot=True,linecolor="blue",lw=0.5)

**Looking for the Missing Values**

In [ ]:
df.isnull().values.any()

In [ ]:
def find_missing_data(data):
    Total = data.isnull().sum().sort_values(ascending = False)
    Percentage = (data.isnull().sum()/data.isnull().count()).sort_values(ascending = False)
    
    return pd.concat([Total,Percentage] , axis = 1 , keys = ['Total' , 'Percent'])

display(find_missing_data(df))

**Missing Value Imputation**

In [ ]:
groupby_country = df.groupby(['Country'])
groupby_country

In [ ]:
def fill_null(series):
    return series.fillna(series.mean())

df["Population"] = groupby_country["Population"].transform(fill_null)
df["HepatitisB"] = groupby_country["HepatitisB"].transform(fill_null)
df["GDP"] = groupby_country["GDP"].transform(fill_null)
df["Total_expenditure"] = groupby_country["Total_expenditure"].transform(fill_null)
df["Alcohol"] = groupby_country["Alcohol"].transform(fill_null)
df["Income_composition_of_resources"] = groupby_country["Income_composition_of_resources"].transform(fill_null)
df["Schooling"] = groupby_country["Schooling"].transform(fill_null)
df["Thinness_1-19_years"] = groupby_country["Thinness_1-19_years"].transform(fill_null)
df["Thinness_5-9_years"] = groupby_country["Thinness_5-9_years"].transform(fill_null)
df["Diphtheria"] = groupby_country["Diphtheria"].transform(fill_null)
df["Polio"] = groupby_country["Polio"].transform(fill_null)
df["Adult_mortality"] = groupby_country["Adult_mortality"].transform(fill_null)
df["Life_expectancy"] = groupby_country["Life_expectancy"].transform(fill_null)


In [ ]:
df.isnull().values.any()

In [ ]:
df.dropna(inplace=True) #There were still missing data fields, that's why null rows are dropped.

In [ ]:
df.isnull().values.any()

**Outliers**

In [ ]:
col_dict = {'Life_expectancy':1 , 'Adult_mortality':2 ,
        'Alcohol':3 , 'Percentage_expenditure': 4, 'HepatitisB': 5,
       'Measles' : 6, 'BMI': 7, 'Under_five_deaths' : 8, 'Polio' : 9, 'Total_expenditure' :10,
       'Diphtheria':11, 'HIV/AIDS':12, 'GDP':13, 'Population' :14,
       'Thinness_1-19_years' :15, 'Thinness_5-9_years' :16,
       'Income_composition_of_resources' : 17, 'Schooling' :18, 'Infant_deaths':19}

In [ ]:
plt.figure(figsize=(20,30))

for variable,i in col_dict.items():
                     plt.subplot(5,4,i)
                     plt.boxplot(df[variable],whis=1.5)
                     plt.title(variable)

plt.show()

Applying Winzorization for Outliers

In [ ]:
plt.figure(figsize=(20,30))

plt.subplot(5,4,1)
winsorized_Life_Expectancy = winsorize(df['Life_expectancy'],(0.01,0))
plt.boxplot(winsorized_Life_Expectancy)
plt.title("winsorized_Life_Expectancy")

plt.subplot(5,4,2)
winsorized_Adult_Mortality = winsorize(df['Adult_mortality'],(0,0.03))
plt.boxplot(winsorized_Adult_Mortality)
plt.title("winsorized_Adult_Mortality")

plt.subplot(5,4,3)
winsorized_Infant_Deaths = winsorize(df['Infant_deaths'],(0,0.11))
plt.boxplot(winsorized_Infant_Deaths)
plt.title("winsorized_Infant_Deaths")

plt.subplot(5,4,4)
winsorized_Alcohol = winsorize(df['Alcohol'],(0,0.01))
plt.boxplot(winsorized_Alcohol)
plt.title("winsorized_Alcohol")

plt.subplot(5,4,5)
winsorized_Percentage_Exp = winsorize(df['Percentage_expenditure'],(0,0.12))
plt.boxplot(winsorized_Percentage_Exp)
plt.title("winsorized_Percentage_Exp")

plt.subplot(5,4,6)
winsorized_HepatitisB = winsorize(df['HepatitisB'],(0.11,0))
plt.boxplot(winsorized_HepatitisB)
plt.title("winsorized_HepatitisB")

plt.subplot(5,4,7)
winsorized_Measles = winsorize(df['Measles'],(0,0.19))
plt.boxplot(winsorized_Measles)
plt.title("winsorized_Measles")

plt.subplot(5,4,8)
winsorized_Under_Five_Deaths = winsorize(df['Under_five_deaths'],(0,0.12))
plt.boxplot(winsorized_Under_Five_Deaths)
plt.title("winsorized_Under_Five_Deaths")

plt.subplot(5,4,9)
winsorized_Polio = winsorize(df['Polio'],(0.09,0))
plt.boxplot(winsorized_Polio)
plt.title("winsorized_Polio")

plt.subplot(5,4,10)
winsorized_Tot_Exp = winsorize(df['Total_expenditure'],(0,0.01))
plt.boxplot(winsorized_Tot_Exp)
plt.title("winsorized_Tot_Exp")

plt.subplot(5,4,11)
winsorized_Diphtheria = winsorize(df['Diphtheria'],(0.11,0))
plt.boxplot(winsorized_Diphtheria)
plt.title("winsorized_Diphtheria")

plt.subplot(5,4,12)
winsorized_HIV = winsorize(df['HIV/AIDS'],(0,0.16))
plt.boxplot(winsorized_HIV)
plt.title("winsorized_HIV")

plt.subplot(5,4,13)
winsorized_GDP = winsorize(df['GDP'],(0,0.13))
plt.boxplot(winsorized_GDP)
plt.title("winsorized_GDP")

plt.subplot(5,4,14)
winsorized_Population = winsorize(df['Population'],(0,0.14))
plt.boxplot(winsorized_Population)
plt.title("winsorized_Population")

plt.subplot(5,4,15)
winsorized_thinness_1to19_years = winsorize(df['Thinness_1-19_years'],(0,0.04))
plt.boxplot(winsorized_thinness_1to19_years)
plt.title("winsorized_thinness_1to19_years")

plt.subplot(5,4,16)
winsorized_thinness_5to9_years = winsorize(df['Thinness_5-9_years'],(0,0.04))
plt.boxplot(winsorized_thinness_5to9_years)
plt.title("winsorized_thinness_5to9_years")

plt.subplot(5,4,17)
winsorized_Income_Comp_Of_Resources = winsorize(df['Income_composition_of_resources'],(0.05,0))
plt.boxplot(winsorized_Income_Comp_Of_Resources)
plt.title("winsorized_Income_Comp_Of_Resources")

plt.subplot(5,4,18)
winsorized_Schooling = winsorize(df['Schooling'],(0.02,0.01))
plt.boxplot(winsorized_Schooling)
plt.title("winsorized_Schooling")

plt.show()

In [ ]:
df['Life_expectancy'] = winsorized_Life_Expectancy
df['Adult_mortality'] = winsorized_Adult_Mortality
df['Infant_deaths'] = winsorized_Infant_Deaths
df['Alcohol'] = winsorized_Alcohol
df['Percentage_expenditure'] = winsorized_Percentage_Exp
df['HepatitisB'] = winsorized_HepatitisB
df['Under_five_deaths'] = winsorized_Under_Five_Deaths
df['Polio'] = winsorized_Polio
df['Total_expenditure'] = winsorized_Tot_Exp
df['Diphtheria'] = winsorized_Diphtheria
df['HIV/AIDS'] = winsorized_HIV
df['GDP'] = winsorized_GDP
df['Population'] = winsorized_Population
df['Thinness_1-19_years'] = winsorized_thinness_1to19_years
df['Thinness_5-9_years'] = winsorized_thinness_5to9_years
df['Income_composition_of_resources'] = winsorized_Income_Comp_Of_Resources
df['Schooling'] = winsorized_Schooling
df['Measles'] = winsorized_Measles

**Visualization**

In [ ]:
df.describe(include= 'O')

In [ ]:
plt.figure(figsize=(6,6))
plt.bar(df.groupby('Status')['Status'].count().index,df.groupby('Status')['Life_expectancy'].mean())
plt.xlabel("Status",fontsize=12)
plt.ylabel("Avg Life_Expectancy",fontsize=12)
plt.title("Life_Expectancy - Status")
plt.show()

In [ ]:
country_and_le = df.groupby('Country')['Life_expectancy'].mean().sort_values(ascending=True)
country_and_le.plot(kind='bar', figsize=(50,15), fontsize=25)
plt.title("Life_Expectancy - Country",fontsize=40)
plt.xlabel("Country",fontsize=35)
plt.ylabel("Avg Life_Expectancy",fontsize=35)
plt.show()

In [ ]:
plt.figure(figsize=(18,40))

plt.subplot(6,3,1)
plt.scatter(df["Life_expectancy"], df["Adult_mortality"])
plt.title("LifeExpectancy vs AdultMortality")

plt.subplot(6,3,2)
plt.scatter(df["Life_expectancy"], df["Infant_deaths"])
plt.title("LifeExpectancy vs Infant_Deaths")

plt.subplot(6,3,3)
plt.scatter(df["Life_expectancy"], df["Alcohol"])
plt.title("LifeExpectancy vs Alcohol")

plt.subplot(6,3,4)
plt.scatter(df["Life_expectancy"], df["Percentage_expenditure"])
plt.title("LifeExpectancy vs Percentage_Exp")

plt.subplot(6,3,5)
plt.scatter(df["Life_expectancy"], df["HepatitisB"])
plt.title("LifeExpectancy vs HepatitisB")

plt.subplot(6,3,6)
plt.scatter(df["Life_expectancy"], df["Under_five_deaths"])
plt.title("LifeExpectancy vs Under_Five_Deaths")

plt.subplot(6,3,7)
plt.scatter(df["Life_expectancy"], df["Polio"])
plt.title("LifeExpectancy vs Polio")

plt.subplot(6,3,8)
plt.scatter(df["Life_expectancy"], df["Total_expenditure"])
plt.title("LifeExpectancy vs Tot_Exp")

plt.subplot(6,3,9)
plt.scatter(df["Life_expectancy"], df["Diphtheria"])
plt.title("LifeExpectancy vs Diphtheria")

plt.subplot(6,3,10)
plt.scatter(df["Life_expectancy"], df["HIV/AIDS"])
plt.title("LifeExpectancy vs HIV")

plt.subplot(6,3,11)
plt.scatter(df["Life_expectancy"], df["GDP"])
plt.title("LifeExpectancy vs GDP")

plt.subplot(6,3,12)
plt.scatter(df["Life_expectancy"], df["Population"])
plt.title("LifeExpectancy vs Population")

plt.subplot(6,3,13)
plt.scatter(df["Life_expectancy"], df["Thinness_1-19_years"])
plt.title("LifeExpectancy vs thinness_1to19_years")

plt.subplot(6,3,14)
plt.scatter(df["Life_expectancy"], df["Thinness_5-9_years"])
plt.title("LifeExpectancy vs thinness_5to9_years")

plt.subplot(6,3,15)
plt.scatter(df["Life_expectancy"], df["Income_composition_of_resources"])
plt.title("LifeExpectancy vs Income_Comp_Of_Resources")

plt.subplot(6,3,16)
plt.scatter(df["Life_expectancy"], df["Schooling"])
plt.title("LifeExpectancy vs Schooling")


plt.show()

**Appliying ttest based on Status**

In [ ]:
stats.ttest_ind(df.loc[df['Status']=='Developed','Life_expectancy'],df.loc[df['Status']=='Developing','Life_expectancy'])

**Categorical Data Conversion**

In [ ]:
df = pd.get_dummies(df,columns=['Status'])

In [ ]:
df.columns.values

In [ ]:
df = df.drop('Country',axis=1)

**Scaling**

In [ ]:
scaler = StandardScaler()

df_scale = pd.DataFrame(scaler.fit_transform(df),columns=df.columns)

In [ ]:
df_scale.head()

**Seperating Dataset into Test and Train**

In [ ]:
y = df['Life_expectancy']
X = df_scale.drop(['Life_expectancy'],axis=1)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state= 42)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

**Appliying ML Models**

**Linear regression**

In [ ]:
lrm = LinearRegression()
lrm.fit(X_train, Y_train) #fit an OLS model

In [ ]:
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)  #making predictions

In [ ]:
print("R-squared of the model in training set is: {}".format(lrm.score(X_train, Y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lrm.score(X_test, Y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(Y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(Y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(Y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((Y_test - y_preds_test) / Y_test)) * 100))

In [ ]:
print(f"Explained Variance Score: {explained_variance_score(y_pred=y_preds_train, y_true=Y_train)}")

**Ridge Regression**

In [ ]:
# using GridSearch for parameter optimization
ridgeregr = GridSearchCV(Ridge(),
                    param_grid={
                        'alpha': [0.01, 0.1, 1]
                    }, verbose=1)

ridgeregr.fit(X_train, Y_train)

ridge = ridgeregr.best_estimator_


In [ ]:
# Making predictions here
y_preds_train = ridge.predict(X_train)
y_preds_test_ridge = ridge.predict(X_test)

print("R-squared of the model in training set is: {}".format(ridge.score(X_train, Y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(ridge.score(X_test, Y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(Y_test, y_preds_test_ridge)))
print("Mean squared error of the prediction is: {}".format(mse(Y_test, y_preds_test_ridge)))
print("Root mean squared error of the prediction is: {}".format(mse(Y_test, y_preds_test_ridge)**(1/2)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((Y_test - y_preds_test_ridge) / Y_test)) * 100))

In [ ]:
print(f"Explained Variance Score: {explained_variance_score(y_pred=y_preds_train, y_true=Y_train)}")

**Lasso Regression**

In [ ]:
# using GridSearch for parameter optimization
lassoregr = GridSearchCV(Lasso(),
                    param_grid={
                        'alpha': [0.01, 0.1, 1]
                    }, verbose=1)

lassoregr.fit(X_train, Y_train)

lasso = lassoregr.best_estimator_

In [ ]:
# We are making predictions here
y_preds_train = lasso.predict(X_train)
y_preds_test_lasso = lasso.predict(X_test)

print("R-squared of the model in training set is: {}".format(lasso.score(X_train, Y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in test set is: {}".format(lasso.score(X_test, Y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(Y_test, y_preds_test_lasso)))
print("Mean squared error of the prediction is: {}".format(mse(Y_test, y_preds_test_lasso)))
print("Root mean squared error of the prediction is: {}".format(rmse(Y_test, y_preds_test_lasso)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((Y_test - y_preds_test_lasso) / Y_test)) * 100))

In [ ]:
print(f"Explained Variance Score: {explained_variance_score(y_pred=y_preds_train, y_true=Y_train)}")

**Random Forest**

In [ ]:
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, Y_train)

In [ ]:
y_pred_random = regressor.predict(X_test)

In [ ]:
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(Y_test, y_pred_random)))
print("Mean squared error of the prediction is: {}".format(mse(Y_test, y_pred_random)))
print("Root mean squared error of the prediction is: {}".format(mse(Y_test, y_pred_random)**(1/2)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((Y_test - y_pred_random) / Y_test)) * 100))

In [ ]:
print(f"Explained Variance Score: {explained_variance_score(y_pred=regressor.predict(X_train), y_true=Y_train)}")

**SVM**

In [ ]:
svr_reg = SVR(kernel='linear', C=1)
svr_reg.fit(X_train, Y_train)
y_predict_svr = svr_reg.predict(X_test)

print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(Y_test, y_predict_svr)))
print("Mean squared error of the prediction is: {}".format(mse(Y_test, y_predict_svr)))
print("Root mean squared error of the prediction is: {}".format(mse(Y_test, y_predict_svr)**(1/2)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((Y_test - y_predict_svr) / Y_test)) * 100))

In [ ]:
print(f"Explained Variance Score: {explained_variance_score(y_pred=regressor.predict(X_train), y_true=Y_train)}")

**XGBOOST**

In [ ]:
xg_reg = GridSearchCV(XGBRegressor(booster='gbtree', subsample=0.75),
                     param_grid={
                         'min_child_weight': [4, 6, 8],
                         'max_depth': [8, 10, 12],
                         'eta': [0.3, 0.03], 
                         'learning_rate': [0.01, 0.1],
                         'reg_alpha': [0.1, 1, 3],
                         'reg_lambda': [0.1, 1, 2, 3]
                     }, 
                     verbose=3, 
                     cv=3)

xg_reg.fit(X_train,Y_train)

In [ ]:
xgb_reg = xg_reg.best_estimator_

In [ ]:
y_predict_xg = xgb_reg.predict(X_test)

print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(Y_test, y_predict_xg)))
print("Mean squared error of the prediction is: {}".format(mse(Y_test, y_predict_xg)))
print("Root mean squared error of the prediction is: {}".format(mse(Y_test, y_predict_xg)**(1/2)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((Y_test - y_predict_xg) / Y_test)) * 100))

In [ ]:
print(f"Explained Variance Score: {explained_variance_score(y_pred=xgb_reg.predict(X_train), y_true=Y_train)}")

**Light GBM**

In [ ]:
d_train = lgb.Dataset(X_train, label=Y_train) #convert the training data to LightGBM data format before training the model

In [ ]:
params_reg = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmsle',
    'max_depth': 6, 
    'learning_rate': 0.1,
    'force_row_wise':True,
    'verbose': -1}
n_estimators = 100

In [ ]:
lgb_reg_model = lgb.train(params_reg, d_train, num_boost_round = 100)
y_predict_lgb_test = lgb_reg_model.predict(X_test)


print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(Y_test, y_predict_lgb_test)))
print("Mean squared error of the prediction is: {}".format(mse(Y_test, y_predict_lgb_test)))
print("Root mean squared error of the prediction is: {}".format(mse(Y_test, y_predict_lgb_test)**(1/2)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((Y_test - y_predict_lgb_test) / Y_test)) * 100))


In [ ]:
print(f"Explained Variance Score: {explained_variance_score(y_pred=lgb_reg_model.predict(X_train), y_true=Y_train)}")

**Conclusion**

Score and RMSE of each model. XGBoost model has the best values.
* Linear Regression:      0.83     |    3.54
* Ridge:                  0.86     |     3.54
* Lasso:                  0.86     |      3.54
* Random Forest:          0.99     |     1.89
* SVM:                    0.99     |    3.59
* XGboost:                0.99     |     1.64
* Light GBM:              0.98     |     1.76
